In [ ]:
import os
from ultralytics import YOLO
import logging
logging.getLogger('matplotlib').setLevel(logging.WARNING)

In [ ]:
# CONSTANTES
BATCH_SIZE = -1
C = 3 # number of channels of the input image
H = 224 # heigh of the input image
W = 224 # width of the input image
current_directory = os.getcwd()

## ONNX: Model conversion

In [ ]:
%run onnx_transform.py --weights weights/yolov11salmons.pt --input_shape $BATCH_SIZE $C $H $W

## TRT fp32

In [ ]:
%run build_trt.py --weights weights/yolov11salmons.onnx  --fp32 --input_shape $BATCH_SIZE $C $H $W --engine_name yolov11salmons_fp32.engine

## TRT fp16

In [ ]:
%run build_trt.py --weights weights/yolov11salmons.onnx  --fp16 --input_shape $BATCH_SIZE $C $H $W --engine_name yolov11salmons_fp16.engine

## TRT int8

In [ ]:
%run build_trt.py --weights weights/yolov11salmons.onnx  --int8 --input_shape $BATCH_SIZE $C $H $W --engine_name yolov11salmons_int8.engine

## validate

In [ ]:
model = YOLO('weights/yolov11salmons.engine', task='segment')
metrics = model.val(data='datasets/salmons/salmons.yaml', imgsz=224, task='segment', verbose=False,conf=0.4,device='cuda',batch=1)
print('base model box mAP50: ', metrics.box.map50)
print('base model box mAP50-95: ', metrics.box.map)